In [1]:
# Workspace problem with several narrow gaps

import torch
import torch.nn as nn

from functools import partial
import numpy as np
from numpy.random import default_rng
from io import StringIO
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter1d
from scipy.stats import gaussian_kde
import matplotlib.animation as animation
import matplotlib.patches as patches
import sys
import time
import cv2
from celluloid import Camera
import time
from IPython.display import HTML

sys.path.append('/home/oscar_palfelt/MSc_thesis/ompl/py-bindings')
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og

rng = default_rng(900)

cuda = False
DEVICE = torch.device("cuda" if cuda else "cpu")

In [2]:
# neural network parameters
mb_size = 256 # mini batch dim
h_Q_dim = 512*6 # encoder dim
h_P_dim = 512*6 # decoder dim

# problem dimenc_dimsions
nDrawnSamples = 49 # number of dependent samples to draw during smapling (length of predicted control series)
nPreviousStates = 1 # number of previous states to condition on the steering prediction
dim = 7 # (truckX, truckY, truckYaw, truckX, trailerY, trailerYaw, steer)
dimGoal = 3
dimObs = 3*2
dimPredict = dim - 0 # predict truck (truckX, truckY, truckYaw, steer)
dataElements = (nDrawnSamples + nPreviousStates) * dim + dimGoal + dimObs # steer sample, current pose (3D: x,y,yaw), goal (3D), dimObs (2D x 3)

z_dim = 3 # latent dim
X_dim = nDrawnSamples*dimPredict
c_dim = dataElements - X_dim # dimension of conditioning variable

gridSize = 11

In [9]:
# define pytorch networks
# based on https://github.com/Jackson-Kang/Pytorch-VAE-tutorial/blob/master/.ipynb_checkpoints/01_Variational_AutoEncoder-checkpoint.ipynb

class Encoder(nn.Module):
    
    def __init__(self, input_dim=X_dim+c_dim, hidden_dim=h_Q_dim, latent_dim=z_dim):
        super(Encoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        self.z_mu = nn.Linear(hidden_dim, latent_dim)
        self.z_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x):

        seq = self.network(x)

        return self.z_mu(seq), self.z_logvar(seq)


class Decoder(nn.Module):
    def __init__(self, latent_dim=z_dim+c_dim, hidden_dim=h_P_dim, output_dim=X_dim):
        super(Decoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        
        return self.network(x)


class NeuralNetwork(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(NeuralNetwork, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
    
    def reparameterization(self, mean, logvar):
        epsilon = torch.randn_like(mean).to(DEVICE)        # sampling epsilon        
        z = mean + torch.exp(0.5 * logvar) * epsilon       # reparameterization trick
        return z

    def forward(self, x, c, encode=True):
        if encode:
            z_mu, z_logvar = self.Encoder(torch.cat((x, c), dim=1))
            z = self.reparameterization(z_mu, z_logvar)

            y = self.Decoder(torch.cat((z, c), dim=1))
            
            return y, z_mu, z_logvar
        else:
            z = x
            y = self.Decoder(torch.cat((z, c), dim=1))    

            return y

network = torch.load('/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/truck_trailer/networks/roadgrid_49.pt', map_location=torch.device(DEVICE))

In [3]:
# read in data from .txt file, re-arrange to allow drawing multiple depedent samples

filename = '/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/truck_trailer/motion_planning/data/ctrlTrailerGrid_27kplans'
rawdata = np.genfromtxt(filename, delimiter=',', dtype='d', usecols=[0,1,2,3,4,5,6,10,11,12,13,14,15,16,17,18]) # disregard init

_, pathsIdx = np.unique(rawdata[:,dim:], axis=0, return_index=True)
pathsIdx.sort()

pathsLengths = np.roll(pathsIdx, -1) - pathsIdx
pathsLengths[-1] = rawdata.shape[0] - np.sum(pathsLengths[:-1])

validLengthsIdx = np.argwhere(pathsLengths >= nPreviousStates + nDrawnSamples) 
validPlansIdx = pathsIdx[validLengthsIdx]

data = np.zeros(shape=(1, dataElements)) # steering angles, samples (x,y,yaw), goal(x,y,yaw)
tempdata = np.copy(data)
for ci, i in enumerate(validPlansIdx[:-1].reshape(-1)):
    for j in range(pathsLengths[ci] - (nPreviousStates + nDrawnSamples - 1)):
        sample = np.arange(start=j, stop=j + nPreviousStates + nDrawnSamples)

        if np.any(np.abs(np.diff(rawdata[i + sample, 0])) > 0.05) or np.any(np.abs(np.diff(rawdata[i + sample, 1])) > 0.05):
            break # ignore duplicate plans (data generation is not perfect)
        
        seqstates = rawdata[i + sample, :dim].reshape(1, (nPreviousStates + nDrawnSamples)*dim)
        # filter out paths that has near zero steering input
        if np.where(np.abs(seqstates[0, 4:(nPreviousStates + nDrawnSamples)*dim:dim]) < 3e-2)[0].shape[0] >= (nDrawnSamples + nPreviousStates) // 1.2:
            continue

        tempdata = np.vstack((tempdata, np.append(seqstates, rawdata[i, dim:].reshape(1, dimGoal+dimObs), axis=1))) 
        # smoothen steering
        tempdata[-1, 4:(nPreviousStates + nDrawnSamples)*dim:dim] = uniform_filter1d(np.poly1d(np.polyfit(np.arange(nPreviousStates + nDrawnSamples), tempdata[-1, 4:(nPreviousStates + nDrawnSamples)*dim:dim], 9))(np.arange(nPreviousStates + nDrawnSamples)), size=7, origin=0, mode='nearest', axis=0)

        if ci % 500 == 0:
            data = np.vstack((data, tempdata[1:]))
            tempdata = np.zeros(shape=(1, dataElements))

data = data[1:]

rng.shuffle(data) # shuffle data (otherwise the data is sorted by obstacles/the map)

pathData = np.copy(data[:, :-dimObs])
obsData = np.copy(data[:, -dimObs:])

numEntries = pathData.shape[0]
print(numEntries)

51202


In [11]:
ratioTestTrain = 0.8;
numTrain = int(numEntries*ratioTestTrain)

nextStates = pathData[:, nPreviousStates*dim:-dimGoal]

prevStatesConditions = pathData[:, :nPreviousStates * dim]
goalStateCondition = pathData[:, -dimGoal:]

conditionsOcc = np.zeros(shape=(numEntries, gridSize ** 2))
for i, row in enumerate(obsData):
    occGrid = np.zeros(shape=[gridSize, gridSize])
    VRT = row[:dimObs//2]
    HOR = row[dimObs//2:]
    occGrid[:, (VRT * (gridSize - 1)).astype(int)] = 1
    occGrid[((1 - HOR) * (gridSize - 1)).astype(int), :] = 1

    conditionsOcc[i] = occGrid.reshape(-1)

cs = np.concatenate((prevStatesConditions, goalStateCondition, conditionsOcc), axis=1)

c_dim = cs.shape[1]
c_train = cs[0:numTrain,:]  
c_test = cs[numTrain:numEntries,:]

In [12]:
# define planning problem
class MyGoal(ob.Goal):
    def __init__(self, si, goal):
        super(MyGoal, self).__init__(si)
        self.goal = goal

    def isSatisfied(self, state):
        return np.linalg.norm(self.goal - np.array([state[0], state[1], (state[2] + np.pi) % (2 * np.pi) - np.pi])) < 0.04


def getThresholdPathLengthObj(si):
     obj = ob.PathLengthOptimizationObjective(si)
     obj.setCostThreshold(ob.Cost(4.0))
     return obj


def isRV4Valid(spaceInformation, state):
    # perform collision checking or check if other constraints are
    # satisfied

    uTruck = int(np.floor(state[0] * gridSize)) # right pointing image axis
    vTruck = int(np.floor(gridSize * (1 - state[1]))) # down pointing image axis

    xTrailer = state[0] - b1 * np.cos(state[2]) - np.linspace(0, b2, 6) * np.cos(state[3])
    yTrailer = state[1] - b1 * np.sin(state[2]) - np.linspace(0, b2, 6) * np.sin(state[3])
    uTrailer = np.clip(np.floor(xTrailer * gridSize), 0, gridSize - 1).astype(int)
    vTrailer = np.clip(np.floor(gridSize * (1 - yTrailer)), 0, gridSize - 1).astype(int)

    #return True
    if spaceInformation.satisfiesBounds(state):
        return occGrid[vTruck,uTruck] > 0 and (occGrid[vTrailer,uTrailer] > 0).all() 


def propagate(start, control, duration, state):
    global nExploredStates
    nExploredStates += 1

    beta = np.arctan(np.tan(control[0]) * a12 / (a11 + a12))

    state[0] = start[0] + v * np.cos(start[2] + beta) * duration
    state[1] = start[1] + v * np.sin(start[2] + beta) * duration
    state[2] = start[2] + v * np.sin(beta) / a12 * duration
    state[3] = start[3] + (v * np.sin(start[2] - start[3]) / (b2 + a21) - v * (b1 - a12) * np.cos(start[2] - start[3]) * np.sin(beta) / (a12 * (b2 + a21))) * duration


def problemDef(sampleCtrl=False):
    # construct the state space we are planning in
    space = ob.RealVectorStateSpace(4)

    # # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(4)
    bounds.setLow(0, 0.001)
    bounds.setLow(1, 0.001)
    bounds.setLow(2, -2 * np.pi)
    bounds.setLow(3, -2 * np.pi)
    bounds.setHigh(0, 0.999)
    bounds.setHigh(1, 0.999)
    bounds.setHigh(2, 2 * np.pi)
    bounds.setHigh(3, 2 * np.pi)
    space.setBounds(bounds)

    # create a control space
    cspace = oc.RealVectorControlSpace(space, 1)

    # set the bounds for the control space
    cbounds = ob.RealVectorBounds(1)
    cbounds.setLow(-0.8)
    cbounds.setHigh(0.8)
    cspace.setBounds(cbounds)

    ss = oc.SimpleSetup(cspace)
    ss.setStatePropagator(oc.StatePropagatorFn(propagate))
    si = ss.getSpaceInformation()

    si.setPropagationStepSize(.07)
    si.setMinMaxControlDuration(1, 1)
    planner = oc.RRT(si)

    ss.setStateValidityChecker(ob.StateValidityCheckerFn( \
        partial(isRV4Valid, ss.getSpaceInformation())))

    ss.setPlanner(planner)
    ss.getProblemDefinition().setOptimizationObjective(getThresholdPathLengthObj(si))
    
    return ss


def plan(planObj, initState, goalState):

    space = ob.RealVectorStateSpace(4)

    start = ob.State(space)
    start[0] = initState[0] # truck x
    start[1] = initState[1] # truck y
    start[2] = initState[2] # truck yaw
    start[3] = initState[3] # trailer yaw

    si = planObj.getSpaceInformation()
    planObj.setStartState(start)
    planObj.setGoal(MyGoal(si, goalState))

    timeTerminationCondition = ob.timedPlannerTerminationCondition(20.0)

    costTerminationCondition = ob.CostConvergenceTerminationCondition(planObj.getProblemDefinition(), solutionsWindow=4, epsilon=1.0)

    solved = planObj.solve(ob.plannerOrTerminationCondition(costTerminationCondition, timeTerminationCondition))


In [13]:
class VehicleState():
    def __init__(self):
        self.x = 0
        self.y = 0
        self.yaw = 0
        self.v = 0

In [14]:
def isCollision(truckState, trailerState):
    uTruck = int(np.floor(truckState.x * gridSize)) # right pointing image axis
    vTruck = int(np.floor(gridSize * (1 - truckState.y))) # down pointing image axis

    xTrailer = truckState.x - b1 * np.cos(truckState.yaw) - np.linspace(0, b2, 6) * np.cos(trailerState.yaw)
    yTrailer = truckState.y - b1 * np.sin(truckState.yaw) - np.linspace(0, b2, 6) * np.sin(trailerState.yaw)
    uTrailer = np.clip(np.floor(xTrailer * gridSize), 0, gridSize - 1).astype(int)
    vTrailer = np.clip(np.floor(gridSize * (1 - yTrailer)), 0, gridSize - 1).astype(int)

    if uTruck >= 0 and uTruck < gridSize and vTruck >= 0 and vTruck < gridSize:
        if not occGrid[vTruck,uTruck] > 0 and (occGrid[vTrailer,uTrailer] > 0).all():
            return True
        return False
    else:
        return True

In [15]:
# plot the latent space
num_viz = 800 # number of samples to draw in latent space

dt = 0.2 # controller time step
v =  0.1 # vehicle speed
L   = 0.05 # vehicle length constant'
W   = 0.023 # truck/trailer width
Lt  = 0.18 # truck length constant
a11 = 0.5 * L
a12 = L - a11
b1  = 0.9 * a12
b2  = 0.5 * Lt 
a21 = Lt - b2

pdef = problemDef()
t = np.tile(np.arange(nDrawnSamples), reps=(num_viz, 1)) * dt

nTests = 100
nCompletedTests = 0
nAttemptedTests = 0
animate = False
CVAEtime = np.zeros(shape=nTests)
RRTtime = np.zeros(shape=nTests)
nCVAEstates = np.zeros(shape=nTests)
nRRTstates = np.zeros(shape=nTests)
nCVAEsuccess = 0
nRRTsuccess = 0
nCVAEfails = 0
nRRTfails = 0
while nCompletedTests < nTests:
    nAttemptedTests += 1

    collision = False

    vizIdx = np.random.randint(numTrain,numEntries-1) # chose a random test scenario
    occGrid = conditionsOcc[vizIdx].reshape(gridSize, gridSize)

    start = pathData[vizIdx, :dim]
    goal = pathData[vizIdx, -dimGoal:]

    truckState = VehicleState()
    truckState.x = start[0]; truckState.y = start[1]; truckState.yaw = start[2]; truckState.v = v

    trailerState = VehicleState()
    trailerState.yaw = start[3]
    trailerState.x = start[5]
    trailerState.y = start[6]

    # check if there is an initial collision. If yes, choose new sample
    if isCollision(truckState, trailerState):
        continue

    steerPrev = prevStatesConditions[vizIdx, 4::dim]
    delta = steerPrev[-1]

    maxSteer = np.max(pathData[:, 4::dim])
    minSteer = np.min(pathData[:, 4::dim])
    steerAngles = np.linspace(minSteer, maxSteer, num=200)

    CVAEstartTime = time.time()
    c_sample_seed = cs[vizIdx,:]
    c_sample = torch.from_numpy(np.repeat([c_sample_seed],num_viz,axis=0)).float().to(DEVICE)
    y_viz = network(torch.randn(num_viz, z_dim).to(DEVICE), c_sample, encode=False).cpu().detach().numpy()

    kdes = np.array([gaussian_kde(y_viz[:, i * dim - 4]) for i in range(1, nDrawnSamples + 1)])

    if animate:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8), gridspec_kw={'width_ratios': [3, 2]})
        camera = Camera(fig)

    for kde in kdes:

        delta = steerAngles[np.argmax(kde.pdf(steerAngles))]

        beta = np.arctan(np.tan(delta) * a12 / (a11 + a12))

        trailerState.x = truckState.x - b1 * np.cos(truckState.yaw) - b2 * np.cos(trailerState.yaw)
        trailerState.y = truckState.y - b1 * np.sin(truckState.yaw) - b2 * np.sin(trailerState.yaw)
        trailerState.yaw += (v * np.sin(truckState.yaw - trailerState.yaw) / (b2 + a21) - v * (b1 - a12) * np.cos(truckState.yaw - trailerState.yaw) * np.sin(beta) / (a12 * (b2 + a21))) * dt

        truckState.x += v * np.cos(truckState.yaw + beta) * dt
        truckState.y += v * np.sin(truckState.yaw + beta) * dt
        truckState.yaw += v * np.sin(beta) / a12 * dt

        # ------------ collision checking ------------
        collision = isCollision(truckState, trailerState)
        if collision: 
            break
        # --------------------------------------------
 
        if animate:
            ax1.scatter(y_viz[:,::dim] * gridSize, y_viz[:,1::dim] * gridSize, color='green', s=70, alpha=0.006, label='CVAE predicted (x,y) states') # nn samples
            ax1.quiver(goal[0] * gridSize, goal[1] * gridSize, np.cos(goal[2]), np.sin(goal[2]), color="blue", scale=11.0, width=0.012, label='Goal pose') # goal
            rectTruck = patches.Rectangle(((truckState.x - np.cos(truckState.yaw) * L/2 + np.sin(truckState.yaw) * W/2) * gridSize, 
                        (truckState.y - np.cos(truckState.yaw) * W/2 - np.sin(truckState.yaw) * L/2) * gridSize), L * gridSize, W * gridSize, angle=np.rad2deg(truckState.yaw), color='purple', alpha=0.8, label='Vehicle')
            rectTrailer = patches.Rectangle(((truckState.x - b1 * np.cos(truckState.yaw) - b2 / 2 * np.cos(trailerState.yaw) - np.cos(trailerState.yaw) * b2/2 + np.sin(trailerState.yaw) * W/2) * gridSize, 
                        (truckState.y - b1 * np.sin(truckState.yaw) - b2 / 2 * np.sin(trailerState.yaw) - np.cos(trailerState.yaw) * W/2 - np.sin(trailerState.yaw) * b2/2) * gridSize), b2 * gridSize, W * gridSize, angle=np.rad2deg(trailerState.yaw), color='orangered', alpha=0.8, label='Trailer')
            ax1.add_patch(rectTruck)
            ax1.add_patch(rectTrailer)
            ax1.imshow(conditionsOcc[vizIdx].reshape(gridSize, gridSize), extent=[0, gridSize, 0, gridSize], cmap='gray')
            ax1.set_xlabel('x', fontsize=12)
            ax1.set_ylabel('y', fontsize=12)
            ax2.plot(steerAngles, kde.pdf(steerAngles), color='C0')
            ax2.set_xlabel('Steering angle', fontsize=12)
            ax2.set_ylabel('Sample density', fontsize=12)
            plt.show()

            camera.snap()

    CVAEendTime = time.time()

    if not collision:
        nCVAEsuccess += 1

        pdef.clear()
        nExploredStates = 0
        RRTstartTime = time.time()
        plan(pdef, initState=start[:4], goalState=[truckState.x, truckState.y, truckState.yaw])
        if pdef.getProblemDefinition().hasExactSolution():
            RRTendTime = time.time()
            RRTtime[nCompletedTests] = RRTendTime - RRTstartTime
            nRRTstates[nCompletedTests] = nExploredStates

            CVAEtime[nCompletedTests] = CVAEendTime - CVAEstartTime
            nCVAEstates[nCompletedTests] = nDrawnSamples # constant

            nRRTsuccess += 1
            nCompletedTests += 1
        else:
            nRRTfails += 1

        if animate:
            fps = 15
            animation = camera.animate(interval=1000/fps)
            print("saving animation")
            animation.save('animation.mp4')
    else:
        nCVAEfails += 1


Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 27833 states
Info:    Solution found in 19.999485 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 24797 states
Info:    Solution found in 20.000102 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 26420 states
Info:    Solution found in 20.000095 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 29088 states
Info:    Solution found in 20.000098 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Addin

         at line 258 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/base/src/Planner.cpp
Error:   RRT: There are no valid initial states!
         at line 109 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/control/planners/rrt/src/RRT.cpp


Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 23881 states
Info:    Solution found in 20.000058 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 26702 states
Info:    Solution found in 20.000081 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 27573 states
Info:    Solution found in 20.000094 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 24359 states
Info:    Solution found in 20.000041 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Addin

         at line 258 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/base/src/Planner.cpp
Error:   RRT: There are no valid initial states!
         at line 109 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/control/planners/rrt/src/RRT.cpp


Debug:   RRT: Discarded start state RealVectorState [0.89008 0.907536 -3.01609 -3.01609]

Info:    No solution found after 0.000240 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 27469 states
Info:    Solution found in 20.000014 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 28767 states
Info:    Solution found in 20.000262 seconds
Debug:   RRT: Discarded start state RealVectorState [0.198561 0.474439 1.43899 1.40342]

Info:    No solution found after 0.000238 seconds


...

         at line 258 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/base/src/Planner.cpp
Error:   RRT: There are no valid initial states!
         at line 109 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/control/planners/rrt/src/RRT.cpp


Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 30701 states
Info:    Solution found in 20.000051 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 21482 states
Info:    Solution found in 20.000183 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 23068 states
Info:    Solution found in 20.000200 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 29019 states
Info:    Solution found in 20.000042 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Addin

         at line 258 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/base/src/Planner.cpp
Error:   RRT: There are no valid initial states!
         at line 109 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/control/planners/rrt/src/RRT.cpp


Debug:   RRT: Discarded start state RealVectorState [0.735708 0.032797 1.4847 1.4847]

Info:    No solution found after 0.000292 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 31941 states
Info:    Solution found in 20.000131 seconds
Info:    RRT: Starting planning with 1 states already in datastructure


KeyboardInterrupt: 

In [24]:
print('Mean CVAE CPU time: {}, std: {}'.format(np.mean(CVAEtime[:nCompletedTests]), np.std(CVAEtime[:nCompletedTests])))
print('Mean RRT CPU time: {}, std: {}'.format(np.mean(RRTtime[:nCompletedTests]), np.std(RRTtime[:nCompletedTests])))

print('Mean CVAE #states: {}, std: {}'.format(np.mean(nCVAEstates[:nCompletedTests]), np.std(nCVAEstates[:nCompletedTests])))
print('Mean RRT #states: {}, std: {}'.format(np.mean(nRRTstates[:nCompletedTests]), np.std(nRRTstates[:nCompletedTests])))

print('CVAE success rate: {}%'.format(nCVAEsuccess / (nCVAEsuccess + nCVAEfails) * 100))
print('RRT success rate: {}%'.format(nRRTsuccess / (nRRTsuccess + nRRTfails) * 100))

Mean CVAE CPU time: 0.1807265111378261, std: 0.06726713163987563
Mean RRT CPU time: 14.943143452916827, std: 3.8521689711811886
Mean CVAE #states: 49.0, std: 0.0
Mean RRT #states: 85963.14285714286, std: 18442.514570792064
CVAE success rate: 26.453143534994066%
RRT success rate: 1.5712682379349048%


In [ ]:
# Mean CVAE CPU time: 0.009101102352142334, std: 0.03087004468798758
# Mean RRT CPU time: 1.2495506715774536, std: 4.345158808266344
# Mean CVAE #states: 3.92, std: 13.293366766925521
# Mean RRT #states: 7303.75, std: 25347.974753962102
# CVAE success rate: 25.83397982932506%
# RRT success rate: 2.4096385542168677%